In [ ]:
import numpy as np

from custom_tree.decision_tree_genetic_algorithm import GeneticDecisionTreeRegressor


class CustomXGBoost:
    def __init__(self, n_estimators=50, learning_rate=0.1, max_depth=3, min_samples_split=10):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        y_pred = np.full_like(y, np.mean(y), dtype=np.float64)
        for _ in range(self.n_estimators):
            residuals = y - y_pred
            tree = GeneticDecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X, residuals)
            update = tree.predict(X)
            y_pred += self.learning_rate * update
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.zeros(X.shape[0])
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        predictions += np.mean([tree.tree['value'] if 'value' in tree.tree else 0 for tree in self.trees])
        return predictions


In [ ]:
import pandas as pd
from sklearn.metrics import r2_score

df_train = pd.read_csv("data/kc_house_data.csv")
df_test = pd.read_csv("data/unseen_kc_house_data.csv")

df_train = df_train.apply(pd.to_numeric, errors='coerce').dropna(axis=1, how='all')
df_test = df_test.apply(pd.to_numeric, errors='coerce').dropna(axis=1, how='all')

y_train = df_train["price"].values
X_train = df_train.drop(columns=["price"]).values

y_test = df_test["price"].values
X_test = df_test.drop(columns=["price"]).values

model = CustomXGBoost(n_estimators=10, learning_rate=0.1, max_depth=3, min_samples_split=5)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = np.mean((y_train - y_train_pred) ** 2)
test_mse = np.mean((y_test - y_test_pred) ** 2)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Train MSE:", train_mse)
print("Test MSE:", test_mse)
print()
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
print()
print("Train R²:", train_r2)
print("Test R²:", test_r2)
